In [10]:
from torch_geometric.explain import Explainer, GNNExplainer

In [11]:
import torch
from hetero_GAT import HeteroGAT

In [15]:

# Load model (EXP 1)
model = HeteroGAT(out_channels=2, num_layers=2, hidden_channels=64)
model.load_state_dict(torch.load('../models/gat_qa_10e_64h_3l.pt', map_location=torch.device('cpu')), strict=False)
model.eval()

RuntimeError: Error(s) in loading state_dict for HeteroGAT:
	size mismatch for convs.0.convs.tag__describes__question.lin_l.weight: copying a param with shape torch.Size([64, 50]) from checkpoint, the shape in current model is torch.Size([384, 50]).
	size mismatch for convs.0.convs.tag__describes__question.lin_l.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([384]).
	size mismatch for convs.0.convs.tag__describes__question.lin_r.weight: copying a param with shape torch.Size([64, 1536]) from checkpoint, the shape in current model is torch.Size([384, 1536]).
	size mismatch for convs.0.convs.tag__describes__answer.lin_l.weight: copying a param with shape torch.Size([64, 50]) from checkpoint, the shape in current model is torch.Size([384, 50]).
	size mismatch for convs.0.convs.tag__describes__answer.lin_l.bias: copying a param with shape torch.Size([64]) from checkpoint, the shape in current model is torch.Size([384]).
	size mism

In [ ]:
from dataset_in_memory import UserGraphDatasetInMemory

train_dataset = UserGraphDatasetInMemory(root="../data", file_name_out='train-4175-qs.pt', question_ids=[])

test_dataset = UserGraphDatasetInMemory(root="../data", file_name_out='test-1790-qs.pt', question_ids=[])

In [ ]:
from torch_geometric.explain import HeteroExplanation

explainer = Explainer(
    model=model,
    algorithm=GNNExplainer(epochs=200),
    explanation_type='model',
    node_mask_type='attributes',
    model_config=dict(
        mode='binary_classification',
        task_level='graph',
        return_type='probs',
    )
)

In [ ]:
from torch_geometric.loader import DataLoader

dataloader = DataLoader(train_dataset, batch_size=1, shuffle=False)
data = next(iter(dataloader))

In [ ]:
post_emb = torch.cat([data.question_emb, data.answer_emb], dim=0)
post_emb.shape

In [ ]:
explanation = explainer(data.x_dict, data.edge_index_dict, batch_dict=data.batch_dict, post_emb=post_emb)
print(f'Generated explanations in {explanation.available_explanations}')

path = 'feature_importance.png'
explanation.visualize_feature_importance(path, top_k=10)
print(f"Feature importance plot has been saved to '{path}'")

path = 'subgraph.pdf'
explanation.visualize_graph(path)
print(f"Subgraph visualization plot has been saved to '{path}'")